Import packages

In [1]:
from __future__ import print_function
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
allow_stdin=True

read the dataset

In [2]:
flag = True
while flag:
    mydataset = input("Please choose the dataset to be imported. The options are mnist & fashion: You chose -> ")
    if mydataset == "mnist":
        from keras.datasets import mnist
        flag = False
    elif mydataset == "fashion":
        from keras.datasets import fashion_mnist
        flag = False
    else:
        print("You did not choose a right dataset to import, maybe next time!")

Please choose the dataset to be imported. The options are mnist & fashion: You chose -> mnist


Using TensorFlow backend.


The following functions are letting the main code to see the other notebooks in the same directory (train, test, model, output)

In [3]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [4]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [5]:
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [6]:
sys.meta_path.append(NotebookFinder())

Set the variables

In [7]:
batch_size = 128
num_classes = 10
epochs = 20
%store num_classes
%store epochs
%store batch_size

Stored 'num_classes' (int)
Stored 'epochs' (int)
Stored 'batch_size' (int)


In [8]:
if mydataset == "mnist":
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
elif mydataset == "fashion":
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
%store x_train
%store y_train
%store x_test
%store y_test

Stored 'x_train' (ndarray)
Stored 'y_train' (ndarray)
Stored 'x_test' (ndarray)
Stored 'y_test' (ndarray)


Train the model by calling the train.ipynb notebook

In [9]:
%run ./train.ipynb

W0723 22:57:12.362266 4643063232 deprecation_wrapper.py:119] From /Users/Toktam/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0723 22:57:12.417139 4643063232 deprecation_wrapper.py:119] From /Users/Toktam/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0723 22:57:12.421503 4643063232 deprecation_wrapper.py:119] From /Users/Toktam/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



60000 train samples
10000 train samples
Stored 'test_data' (ndarray)
Stored 'test_label' (ndarray)


W0723 22:57:12.477925 4643063232 deprecation_wrapper.py:119] From /Users/Toktam/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0723 22:57:12.616919 4643063232 deprecation.py:506] From /Users/Toktam/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0723 22:57:12.750693 4643063232 deprecation_wrapper.py:119] From /Users/Toktam/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Stored 'model' (Sequential)


/Users/Toktam/anaconda3/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
W0723 22:57:13.178049 4643063232 deprecation_wrapper.py:119] From /Users/Toktam/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0723 22:57:13.330455 4643063232 deprecation.py:323] From /Users/Toktam/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 128 samples, validate on 10000 samples
Epoch 1/20
128/128 [==============================] - 1s 6ms/step - loss: 2.3581 - acc: 0.0938 - val_loss: 2.1595 - val_acc: 0.2758
Epoch 2/20
128/128 [==============================] - 0s 2ms/step - loss: 1.6164 - acc: 0.4531 - val_loss: 1.7409 - val_acc: 0.5356
Epoch 3/20
128/128 [==============================] - 0s 2ms/step - loss: 1.3611 - acc: 0.7656 - val_loss: 1.4430 - val_acc: 0.5362
Epoch 4/20
128/128 [==============================] - 0s 2ms/step - loss: 0.7542 - acc: 0.8125 - val_loss: 1.2579 - val_acc: 0.6340
Epoch 5/20
128/128 [==============================] - 0s 2ms/step - loss: 0.4958 - acc: 0.9453 - val_loss: 1.2518 - val_acc: 0.5971
Epoch 6/20
128/128 [==============================] - 0s 2ms/step - loss: 0.4475 - acc: 0.8516 - val_loss: 1.4195 - val_acc: 0.5667
Epoch 7/20
128/128 [==============================] - 0s 3ms/step - loss: 0.3603 - acc: 0.9297 - val_loss: 1.2193 - val_acc: 0.6397
Epoch 8/20
128/128 [========

Test the model by calling the test.ipynb notebook

In [10]:
%run ./test.ipynb
%store -r test_score

Test loss: 0.9824548957586289
Test accuracy: 0.7221
Stored 'test_score' (list)


In [11]:
%run ./output.ipynb

[0.11718455487646338, 0.9841] from output


In [12]:
print("Finished!")

Finished!
